In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import copy
import h5py
import toml
import glob
import sleap_io as sio
from tqdm.auto import tqdm

# Get counts for SLEAP training

In [3]:
root_dir = os.path.expanduser("~/shared_folder/active_lab_members/markowitz_jeffrey/active_projects")

In [4]:
# fused keypoint location...
fused_keypoint_files = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/_labels/basler-nir-plexiglass-arena-keypoints-fused-round2_weights-None_bpass-None.slp")
f"N(fused frames) {len(sio.load_slp(fused_keypoint_files))}"

'N(fused frames) 862'

In [5]:
# how big is the whole dataset??
"keypoints_basler_nir_plexiglass_arena/sleap_training_round2/_labels/basler-nir-plexiglass-arena-keypoints-fused-kneejoints_weights-(0.0, 1.0)_bpass-None_fluo-aligned.slp"

'keypoints_basler_nir_plexiglass_arena/sleap_training_round2/_labels/basler-nir-plexiglass-arena-keypoints-fused-kneejoints_weights-(0.0, 1.0)_bpass-None_fluo-aligned.slp'

In [6]:
# fused keypoint location...
fused_keypoint_files = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/_labels/basler-nir-plexiglass-arena-keypoints-fused-kneejoints_weights-(0.0, 1.0)_bpass-None_fluo-aligned.slp")
f"N(fused frames, kneejoint) {len(sio.load_slp(fused_keypoint_files))}"

'N(fused frames, kneejoint) 872'

# Autolabeled data

In [7]:
autolabel_keypoint_metadata_file = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/_labels_qd/kpoint_training_dataset_qd_alignment_round2_manual_labeling_nofluo_version-v1-embed-False.toml")
autolabel_metadata = toml.load(autolabel_keypoint_metadata_file)

In [8]:
f"N(autolabeled frames, standard)={len(autolabel_metadata['frames'])}"

'N(autolabeled frames, standard)=114629'

In [9]:
# count for each camera

In [10]:
camera_per_frame = [_["camera"] for _ in autolabel_metadata["frames"]]

In [11]:
counts = {}
for camera in camera_per_frame:
    counts[camera]  = counts.get(camera, 0) + 1

In [12]:
counts_heldout = {}
for camera in counts.keys():
    counts_heldout[camera] = 0
    for camera2 in counts.keys():
        if camera2 != camera:
            counts_heldout[camera] += counts[camera2]

In [13]:
counts

{'Basler-267601741000-24383488': 21866,
 'Basler-267601746219-24404505': 25022,
 'Basler-267601746223-24404515': 18953,
 'Basler-26760175421F-24461855': 24343,
 'Basler-2676017611A4-24514980': 24445}

In [14]:
counts_heldout

{'Basler-267601741000-24383488': 92763,
 'Basler-267601746219-24404505': 89607,
 'Basler-267601746223-24404515': 95676,
 'Basler-26760175421F-24461855': 90286,
 'Basler-2676017611A4-24514980': 90184}

In [15]:
# number of mice from each group...


In [16]:
joint = False
if joint:
    kpoint_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/keypoint_inference_kneejoints_export_fused_weights-None_bpass-None")
else:
    kpoint_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/keypoint_inference_export_fused_weights-None_bpass-None")

In [17]:
sleap_files = sorted(glob.glob(os.path.join(kpoint_dir, "**", "*.slp"), recursive=True))

In [18]:
total_frames = 0

In [19]:
frame_counts = [len(sio.load_slp(_sleap_file).labeled_frames) for _sleap_file in tqdm(sleap_files)]

  0%|          | 0/172 [00:00<?, ?it/s]

In [20]:
sum(frame_counts)

1425041

# Autolabeled data (kneejoints)

In [21]:
autolabel_keypoint_metadata_file = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/_labels_qd/kpoint_training_dataset_qd_alignment_kneejoints_reflect_only_round2_manual_labeling_nofluo_version-v1-embed-False.toml")
autolabel_metadata = toml.load(autolabel_keypoint_metadata_file)

In [22]:
f"N(autolabeled frames, kneejoints)={len(autolabel_metadata['frames'])}"

'N(autolabeled frames, kneejoints)=17901'

# Segmentation model

In [25]:
import joblib

In [26]:
segmentation_data = "/storage/coda1/p-jmarkowitz30/0/shared/active_lab_members/markowitz_jeffrey/active_projects/segmentation_basler_nir_plexiglass_arena/segmentation_training_data.pkl.gz"

In [27]:
training_data = joblib.load(segmentation_data)

In [28]:
training_datapoints = training_data["x"].shape[0]

In [29]:
print(training_datapoints)

1395
